In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#Importing Librabries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

# TIME SERIES
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

# warnings 
import warnings
warnings.filterwarnings("ignore")

In [ ]:
Path = '../input/competitive-data-science-predict-future-sales/'
shops = pd.read_csv(Path +'shops.csv')
sales_train = pd.read_csv(Path+'sales_train.csv',parse_dates=True)
item_categories = pd.read_csv(Path+'item_categories.csv')
test = pd.read_csv(Path+'test.csv')
item = pd.read_csv(Path+'items.csv')

**Data fields**

ID - an Id that represents a (Shop, Item) tuple within the test set

shop_id - unique identifier of a shop

item_id - unique identifier of a product

item_category_id - unique identifier of item category

item_cnt_day - number of products sold. You are predicting a monthly amount of this measure

item_price - current price of an item

date - date in format dd/mm/yyyy

date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33

item_name - name of item

shop_name - name of shop

item_category_name - name of item category

In [ ]:
print(shops.shape,sales_train.shape,item_categories.shape , test.shape,item.shape)

In [ ]:
by_cat = item.groupby(['item_category_id']).size().sort_values(ascending=False).head(20).plot.bar()
plt.xlabel('Items per category')
plt.ylabel('No. of Times')

In [ ]:
total_sale = sales_train.groupby(['date_block_num'])['item_cnt_day'].sum()
#total_sale.head()
plt.figure(figsize=(8,8))
plt.title('Total sales of company')
plt.xlabel('Time (January 2013 is 0, February 2013 is 1,..., October 2015 is 33)')
plt.ylabel('sales')
plt.plot(total_sale)

A rolling analysis of a time series model is often used to assess the model’s stability over time. When analyzing financial time series data using a statistical model, a key assumption is that the parameters of the model are constant over time. However, the economic environment often changes considerably, and it may not be reasonable to assume that a model’s parameters are constant. A common technique to assess the constancy of a model’s parameters is to compute parameter estimates over a rolling window of a fixed size through the sample. If the parameters are truly constant over the entire sample, then the estimates over the rolling windows should not be too different. If the parameters change at some point during the sample, then the rolling estimates should capture this instability.

In [ ]:
#checking timeseries stationarity 
#1. plotting rolling statistics curve
#2. dickey fuller test 
plt.figure(figsize=(16,6))
plt.plot(total_sale, color='blue',label='Original')
plt.plot(total_sale.rolling(window= 12,center= False).mean(),label='Rolling Mean')
plt.plot(total_sale.rolling(window=12,center= False).std(),label='Rolling std')
plt.legend()

In [ ]:
sales_train.head()

In [ ]:
#formatting the date column correctly
#sales_train.date=sales_train.date.apply(lambda x:datetime.datetime.strptime(x, '%d.%m.%Y'))

sales_train['date'] = pd.to_datetime(sales_train['date'], format="%d.%m.%Y")
sales_train['DayOfWeekNum'] = sales_train['date'].dt.dayofweek
sales_train['DayOfWeek'] = sales_train['date'].dt.weekday_name
sales_train['MonthDayNum'] = sales_train['date'].dt.day

**Checking Trends, seasionality and residuals **

Time series datasets may contain trends and seasonality, which may need to be removed prior to modeling.

Trends can result in a varying mean over time, whereas seasonality can result in a changing variance over time,

**Additive or multiplicative?**

It’s important to understand what the difference between a multiplicative time series and an additive one before we go any further.

There are three components to a time series:

– trend how things are overall changing

– seasonality how things change within a given period e.g. a year, month, week, day

– error/residual/irregular activity not explained by the trend or the seasonal value

How these three components interact determines the difference between a multiplicative and an additive time series.

In a multiplicative time series, the components multiply together to make the time series. If you have an increasing trend, the amplitude of seasonal activity increases. Everything becomes more exaggerated. This is common when you’re looking at web traffic.

In an additive time series, the components add together to make the time series. If you have an increasing trend, you still see roughly the same size peaks and troughs throughout the time series. This is often seen in indexed time series where the absolute value is growing but changes stay relative.

In [ ]:
import statsmodels.api as sm
tse = sm.tsa.seasonal_decompose(total_sale.values,freq=12,model='multiplicative')
tse.plot()

we assume an additive model, then we can write

**yt=St+Tt+Et**

where yt is the data at period t, St is the seasonal component at period t, Tt is the trend-cycle component at period tt and Et is the remainder (or irregular or error) component at period t Similarly for Multiplicative model,

**yt=St x Tt x Et**

In [ ]:
tse = sm.tsa.seasonal_decompose(total_sale.values,freq=12,model='additive')
tse.plot()

**Stationarity **

**Stationary Time Series:**

data does not have any upward or downward trend or seasonal effects. Mean or variance are consistent over time.


![](http://miro.medium.com/max/454/1*jNeo1bAgBOS_FvyUhDi-0g.png)

**Non-Stationary Time Series:**

data show trends, seasonal effects, and other structures depend on time. Forecasting performance is dependent on the time of observation. Mean and variance change over time and a drift in the model is captured.

![](https://miro.medium.com/max/411/1*-iOconMEaLW5ttxhHCdKBw.png)

**Augmented Dickey-Fuller test (ADF)**

ADF tests the null hypothesis that a unit root is present in time series sample. ADF statistic is a negative number and more negative it is the stronger the rejection of the hypothesis that there is a unit root.

We interpret this result using the p-value from the test. A p-value below a threshold (such as 5% or 1%) suggests we reject the null hypothesis (stationary), otherwise a p-value above the threshold suggests we fail to reject the null hypothesis (non-stationary).

**Null Hypotehsis (H0): **

If accepted, it suggests the time series has a unit root, meaning it is non-stationary. It has some time dependent structure.

**Alternate Hypothesis (H1):** 

The null hypothesis is rejected; it suggests the time series does not have a unit root, meaning it is stationary.

**p-value > 0.05: **

Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.

**p-value <= 0.05: **

Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

**means if null hypothesis(H0) accepted, p>0.05 then series is not stationary **

In [ ]:
# Stationarity tests
def test_stationarity(timeseries):
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

test_stationarity(total_sale)

Here we can see that statistic is greater than all critical values ,means we can accept null hypothesis from both statement we can conclude that series is not stationary . we have convert the series into stationary series. 


>** Difference Transform**

Differencing is a method of transforming a time series dataset.

Differencing is performed by subtracting the previous observation from the current observation.

**difference(t) = observation(t) - observation(t-1)**

Inverting the process is required when a prediction must be converted back into the original scale.

This process can be reversed by adding the observation at the prior time step to the difference value.

**inverted(t) = differenced(t) + observation(t-1)**

In this way, a series of differences and inverted differences can be calculated.

**Lag Difference**

Taking the difference between consecutive observations is called a lag-1 difference.

The lag difference can be adjusted to suit the specific temporal structure.

For time series with a seasonal component, the lag may be expected to be the period (width) of the seasonality.

**Difference Order**

Some temporal structure may still exist after performing a differencing operation, such as in the case of a nonlinear trend.

As such, the process of differencing can be repeated more than once until all temporal dependence has been removed.

The number of times that differencing is performed is called the difference order.

In [ ]:
# to remove trend
from pandas import Series as Series
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

# invert differenced forecast
def inverse_difference(last_ob, value):
    return value + last_ob

In [ ]:
new_ts1=difference(total_sale,12) 

In [ ]:
new_ts=difference(total_sale)

fig, axs = plt.subplots(3,figsize=(16,16))
axs[0].plot(total_sale)
axs[0].set_title('original')

axs[1].plot(new_ts)
axs[1].set_title('After De-trend')

axs[2].plot(new_ts1)# assuming the seasonality is 12 months long
axs[2].set_title('After De-serialization')


In [ ]:
# now testing the stationarity again after de-seasonality
test_stationarity(new_ts1)

Now after the transformations, our p-value for the DF test is well within 5 %. Hence we can assume Stationarity of the series.

We can easily get back the original series using the inverse transform function that we have defined above.

** Introduction to ARMA Time Series Modeling**

ARMA models are commonly used in time series modeling. In ARMA model, AR stands for auto-regression and MA stands for moving average. If these words sound intimidating to you, worry not – I’ll simplify these concepts in next few minutes for you!

We will now develop a knack for these terms and understand the characteristics associated with these models. But before we start,** you should remember, AR or MA are not applicable on non-stationary series**

A pure Auto Regressive (AR only) model is one where Yt depends only on its own lags. That is, Yt is a function of the ‘lags of Yt’.

![](https://www.machinelearningplus.com/wp-content/uploads/2019/02/Equation-1-min.png)

where, $Y{t-1}$ is the lag1 of the series, $\beta1$ is the coefficient of lag1 that the model estimates and $\alpha$ is the intercept term, also estimated by the model.

Likewise a pure Moving Average (MA only) model is one where Yt depends only on the lagged forecast errors.

![](https://www.machinelearningplus.com/wp-content/uploads/2019/02/Equation-2-min.png)

An ARIMA model is one where the time series was differenced at least once to make it stationary and you combine the AR and the MA terms. So the equation becomes:

![](https://www.machinelearningplus.com/wp-content/uploads/2019/02/Equation-4-min-1024x91.png)



ARIMA model in words:


Predicted Yt = Constant + Linear combination Lags of Y (upto p lags) + Linear Combination of Lagged forecast errors (upto q lags)


An ARIMA model is characterized by 3 terms: p, d, q

where,

**1. Number of AR (Auto-Regressive) terms (p): **AR terms are just lags of dependent variable. For instance if p is 5, the predictors for x(t) will be x(t-1)….x(t-5).

**2. Number of MA (Moving Average) terms (q):**   MA terms are lagged forecast errors in prediction equation. For instance if q is 5, the predictors for x(t) will be e(t-1)….e(t-5) where e(i) is the difference between the moving average at ith instant and actual value.

**3. Number of Differences (d):**  These are the number of nonseasonal differences, i.e. in this case we took the first order difference. So either we can pass that variable and put d=0 or pass the original variable and put d=1. Both will generate same results

An importance concern here is how to determine the value of ‘p’ and ‘q’. We use two plots to determine these numbers. Lets discuss them first.

**Autocorrelation Function (ACF): **

It is a measure of the correlation between the the TS with a lagged version of itself. For instance at lag 5, ACF would compare series at time instant ‘t1’…’t2’ with series at instant ‘t1-5’…’t2-5’ (t1-5 and t2 being end points).

**Partial Autocorrelation Function (PACF):**

This measures the correlation between the TS with a lagged version of itself but after eliminating the variations already explained by the intervening comparisons. Eg at lag 5, it will check the correlation but remove the effects already explained by lags 1 to 4.

In [ ]:
lag_acf = acf(total_sale, nlags=20)
lag_pacf = pacf(total_sale, nlags=20, method='ols')

In [ ]:
#Plot ACF: 
plt.subplot(121) 
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(total_sale)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(total_sale)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')
#Plot PACF:
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(total_sale)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(total_sale)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

In this plot, the two dotted lines on either sides of 0 are the confidence interevals. These can be used to determine the ‘p’ and ‘q’ values as:

p – The lag value where the PACF chart crosses the upper confidence interval for the first time. If you notice closely, in this case p=2.

q – The lag value where the ACF chart crosses the upper confidence interval for the first time. If you notice closely, in this case q=2.

Now, lets make 3 different ARIMA models considering individual as well as combined effects. I will also print the RSS for each. Please note that here RSS is for the values of residuals and not actual series.

We need to load the ARIMA model first:

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

The p,d,q values can be specified using the order argument of ARIMA which take a tuple (p,d,q). Let model the 3 cases:

In [ ]:
model = ARIMA(total_sale, order=(2, 1, 0))  
results_AR = model.fit(disp=-1)  
plt.plot(total_sale)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_AR.fittedvalues-total_sale)**2))